In [38]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn import preprocessing
from scipy.spatial import distance
from scipy.stats import spearmanr
from sklearn.metrics.pairwise import cosine_similarity
sns.set_theme(style="whitegrid")

In [39]:
df = pd.read_csv('data.csv')
#df.sort_values(by=['Average'],ascending=False, inplace=True)
#df.drop(df.tail(3).index,inplace=True)
df.head()

,Name,Brewery,Adam,Dad,Alan,Dave,Fi,Average,Std,Style,Abv,Hops,Grains,IBU
0,Swan Gold,Swan,6.5,7.5,8.0,6.0,7.5,7.1,0.82,Golden Ale,4.0%,Admiral;Saaz;Styrian Goldings,Pale Malt; Wheat; Crystal,NaN
1,Winter Swanerland,Swan,5.0,5.0,6.0,7.0,7.0,6.0,1.00,Ruby Ale,4.8%,First Gold;Endeavour,Pale Maris Otter; Crystal; Wheat; Chocolate.,NaN
2,Mutation IPA,Swan,6.0,5.5,5.0,8.0,6.5,6.2,1.15,IPA,5.5%,Pilgrim;Citra;Comet,Pale Malt; Wheat; Crystal,NaN
3,Arm Breaker,Swan,3.0,4.5,5.0,6.0,5.0,4.7,1.10,IPA,6.0%,Goldings;Admiral,Pale; Wheat; Crystal,NaN
4,Black Swan,Swan,6.0,4.0,4.0,8.5,9.0,6.3,2.39,Stout,4.5%,First Gold,Chocolate; Crystal; Pale Malts; Roasted Barley...,NaN


Normalise Induvidual Rating

In [40]:
names = ['Adam', 'Dad', 'Alan', 'Dave', 'Fi']
for name in names:
    x = df[name].values.reshape(-1,1)
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    x_scaled = np.subtract(x_scaled, np.mean(x_scaled)-0.5)
    df[name] = x_scaled*1

In [41]:
dictionary = {}
for i in range(np.shape(names)[0]):
    for k in range(i+1, np.shape(names)[0]):
        p1 = df[names[i]].values.reshape(-1,1)
        p2 = df[names[k]].values.reshape(-1,1)
        dictionary[names[i]+' And '+names[k]] = distance.euclidean(p1, p2)
dictionary = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=False))
dictionary

{'Dave And Fi': 1.3623990620731405,
 'Adam And Dave': 1.8430115668263398,
 'Adam And Fi': 1.9382828196977915,
 'Adam And Alan': 1.9476545643728373,
 'Adam And Dad': 1.991401616826858,
 'Dad And Alan': 2.034422867668957,
 'Alan And Fi': 2.490095799219234,
 'Dad And Dave': 2.58237205746887,
 'Dad And Fi': 2.589141558966253,
 'Alan And Dave': 2.620981556516661}

In [42]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

dictionary = {}
for i in range(np.shape(names)[0]):
    for k in range(i+1, np.shape(names)[0]):
        p1 = df[names[i]].values
        p2 = df[names[k]].values
        dictionary[names[i]+' And '+names[k]] = cosine_similarity(p1, p2)
dictionary = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=True))
dictionary

{'Dave And Fi': 0.9627897487351427,
 'Adam And Dave': 0.9321973643971697,
 'Adam And Alan': 0.9276350289102261,
 'Adam And Fi': 0.9253121509786544,
 'Adam And Dad': 0.9251425153719537,
 'Dad And Alan': 0.923531565561907,
 'Alan And Fi': 0.8810639439443787,
 'Dad And Dave': 0.8728957087744471,
 'Dad And Fi': 0.8726024698709952,
 'Alan And Dave': 0.867734795170661}

In [43]:
dictionary = {}
for i in range(np.shape(names)[0]):
    for k in range(i+1, np.shape(names)[0]):
        p1 = df[names[i]].values.reshape(-1,1)
        p2 = df[names[k]].values.reshape(-1,1)
        dictionary[names[i]+' And '+names[k]] = spearmanr(p1.argsort(axis=0), p2.argsort(axis=0))[0]
dictionary = dict(sorted(dictionary.items(), key=lambda item: item[1], reverse=True))
dictionary

{'Alan And Fi': 0.25005107252298264,
 'Adam And Alan': 0.08981954375212803,
 'Adam And Dad': -0.02008852570650324,
 'Alan And Dave': -0.026540687776642834,
 'Dave And Fi': -0.042798774259448426,
 'Dad And Dave': -0.04475655430711611,
 'Adam And Fi': -0.04695267279536944,
 'Dad And Fi': -0.07875383043922371,
 'Dad And Alan': -0.09950629894450122,
 'Adam And Dave': -0.10309839972761321}